In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
from functools import partial

from dotenv import load_dotenv

from optimization import jg_opt
from pisa.administrative_area import AdministrativeArea
from pisa.facilities import Facilities
from pisa.population import WorldpopPopulation
from pisa.population_served_by_isopolygons import get_population_served_by_isopolygons

# keep the API keys in a `.env` file in the local root directory
load_dotenv()


True

Some choices:

- Population with WorldPop or Facebook
- isopolygon calculator either Mapbox or OSM

### Define Administrative Area

Let op: the administrative area is a country because of the package that we use for fetching the data (gadm).

The naming is confusing: we want the administrative area to be a subset of a country (in this case, the administrative area should be Baucau). 

Should be easy to fix once we change gadm to pygadm (see issue #59 on GitHub)

In [3]:
timor_leste = AdministrativeArea(country_name="Timor-Leste", admin_level=1)

# these are the boundaries of Baucau
# type: Polygon
baucau = timor_leste.get_admin_area_boundaries("Baucau")

INFO:pisa.administrative_area:Validating country name: Timor-Leste
INFO:pisa.administrative_area:Country name 'Timor-Leste' validated successfully
INFO:pisa.administrative_area:Retrieving boundaries of all administrative areas of level 1 for country Timor-Leste


### Get population 
In this example, from WorldPop

In [4]:
population_gdf = WorldpopPopulation(
    admin_area_boundaries=baucau, iso3_country_code=timor_leste.get_iso3_country_code()
).get_population_gdf()

population_count = population_gdf.population.values

### Create instance of Facilities class for Baucau

In [6]:
facilities_calculator = Facilities(admin_area_boundaries=baucau)

## Calculate isopolygons

In [7]:
DISTANCE_TYPE = "length"

DISTANCE_VALUES = [2000, 5000, 10000]

MODE_OF_TRANSPORT = "driving"

#### Using Mapbox API

In [8]:
from pisa.isopolygons import MapboxIsopolygonCalculator

MAPBOX_API_TOKEN = os.getenv("MAPBOX_API_TOKEN")

#### Calculate isopolygons for existing facilities

In [9]:
isopolygons_existing_facilities = MapboxIsopolygonCalculator(
    facilities_df=facilities_calculator.get_existing_facilities(),
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    mode_of_transport=MODE_OF_TRANSPORT,
    mapbox_api_token=MAPBOX_API_TOKEN,
).calculate_isopolygons()

population_served_current = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_existing_facilities
)

current = {DISTANCE_TYPE: population_served_current}

INFO:pisa.facilities:Retrieving existing facilities with tags {'amenity': 'hospital', 'building': 'hospital'} using OSM.
INFO:pisa.facilities:Successfully retrieved existing facilities from OSM.


#### Calculate isopolygons for potential facilities

In [10]:
isopolygons_potential_facilities = MapboxIsopolygonCalculator(
    facilities_df=facilities_calculator.estimate_potential_facilities(),
    distance_type=DISTANCE_TYPE,
    distance_values=DISTANCE_VALUES,
    mode_of_transport=MODE_OF_TRANSPORT,
    mapbox_api_token=MAPBOX_API_TOKEN,
).calculate_isopolygons()

population_served_potential = get_population_served_by_isopolygons(
    grouped_population=population_gdf, isopolygons=isopolygons_potential_facilities
)

potential = {DISTANCE_TYPE: population_served_potential}

### Calculate solutions

In [11]:
BUDGET = [
    5,
    20,
    50,
]  # budget for the optimization in terms of how many locations can be built


CBC_SOLVER_PATH = os.getenv(
    "CBC_SOLVER_PATH"
)  # path to the cbc executable (e.g. /opt/homebrew/bin/cbc)

cbc_optimize = partial(jg_opt.OpenOptimize, solver_path=CBC_SOLVER_PATH)

values, solutions = jg_opt.Solve(
    household=population_count,
    current=current,
    potential=potential,
    accessibility=DISTANCE_TYPE,
    budgets=BUDGET,
    optimize=cbc_optimize,
    type="ID",
)

In [12]:
values

,10000,5000,2000
5,0.474183,0.291701,0.142469
20,0.492997,0.315200,0.149626
50,0.497300,0.324737,0.153492


In [13]:
solutions

,10000,5000,2000
5,"[28, 22, 27, 39, 18]","[25, 22, 39, 27, 28]","[39, 20, 27, 7, 25]"
20,"[28, 22, 27, 39, 18, 10, 25, 20, 29, 51, 16, 8...","[25, 22, 39, 27, 28, 43, 16, 26, 8, 36, 23, 18...","[39, 20, 27, 7, 25, 43, 10, 13, 6, 22, 42, 26,..."
50,"[28, 22, 27, 39, 18, 10, 25, 20, 29, 51, 16, 8...","[25, 22, 39, 27, 28, 43, 16, 26, 8, 36, 23, 18...","[39, 20, 27, 7, 25, 43, 10, 13, 6, 22, 42, 26,..."
